# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import os
from dotenv import load_dotenv


load_dotenv() # loads .env into the environment

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY is not set")

In [2]:
from openai import OpenAI

client = OpenAI() # automatically reads OPENAI_API_KEY


In [3]:

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course! Alas, a classic jest to bring mirth to one's heart!


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, I don't have access to personal information like your name. But you can always tell me your name if you'd like!


In [8]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [16]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': "#222020"})))
 
    return pn.Column(*panels)


In [17]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'38770c1c-09ee-4b73-b35f-e60e89e8426d': {'version…

In [12]:
# sending the memory back
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "cheese pizza",
    "size": "medium"
  },
  "toppings": [
    "sausage",
    "canadian bacon",
    "AI sauce"
  ],
  "drinks": [
    {
      "type": "sprite",
      "size": "large"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "small"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Movie Ticket booking bot

In [20]:
def collect_messages(_):
    user_input = inp.value

    context.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context
    )

    assistant_message = response.choices[0].message.content
    context.append({"role": "assistant", "content": assistant_message})

    return pn.pane.Markdown(assistant_message)

In [23]:
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are MovieBot, an automated service to collect orders for movie ticket theater. \
You first greet the customer, then collect the order, \
Each movie has three ticket types: \
Adult (9.00) \
Student (6.50) \
Child (4.95) \
Always ask which ticket type and how many tickets \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all movie options and extras to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Hamnet 9.00, 6.50, 4.95 \
Sentimental Value 9.00, 6.50, 4.95 \
Marty Supreme 9.00, 6.50, 4.95 \
Extras: \
Popcorn 5.00, 3.75, 2.95 \
Nachos 3.50, 1.95 \
M&M box 3.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation.param.clicks)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'7d5cd4fb-534f-4469-a1f0-bc6c44c1d40a': {'version…

In [24]:
# sending the memory back
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous order. Itemize the price for each item\
 The fields should be 1) movie tickets, include type of ticket 2) list of extras 3) list of drinks, include size   4)total price '},    
)
 
response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "movie_tickets": [
        {"ticket_type": "Adult", "price": 9.00},
        {"ticket_type": "Adult", "price": 9.00},
        {"ticket_type": "Student", "price": 6.50}
    ],
    "extras": [
        {"item": "Large Popcorn", "price": 5.00},
        {"item": "Large Popcorn", "price": 5.00},
        {"item": "M&M Box", "price": 3.00}
    ],
    "drinks": [
        {"drink": "Large Sprite", "price": 3.00},
        {"drink": "Large Sprite", "price": 3.00}
    ],
    "total_price": 52.00
}
```


### Expenses Reimbursement Bot

- Collects expense items from the user

- Trusts the user’s stated prices and totals

- Approves reimbursement if the “total looks reasonable”

In [25]:
# V1: Trusts everything

pn.extension()

panels = []

context = [{'role':'system', 'content':"""
You are ExpenseBot, an automated assistant that helps employees submit expense reimbursement requests.

You politely greet the user and ask them to list their expenses.
For each expense, ask for:
- item description
- price

You trust the prices and totals provided by the user.
Do not question or verify calculations.
If the user provides a total, assume it is correct.

Once the user says they are done:
- summarize all expenses
- restate the total amount
- approve the reimbursement

Be friendly, supportive, and efficient.
Assume the user is honest and accurate.
Do not challenge or double-check numbers.
"""}]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation.param.clicks)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'9d4ba3f1-ec53-4211-acb0-27d8f02931de': {'version…

In [26]:
# sending the memory back
messages =  context.copy()
messages.append({
  "role": "system",
  "content": """
Based on the full conversation above, output ONLY a valid JSON object.
Do not include markdown, backticks, or explanatory text.

The JSON MUST follow this exact schema:

{
  "expenses": [
    {
      "description": "string",
      "category": "string",
      "unit_price": 0,
      "quantity": 0,
      "reported_total": 0
    }
  ],
  "reported_grand_total": 0,
  "currency": "USD",
  "approval_status": "approved | flagged | rejected",
  "notes": "string"
}
"""
})
 
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "expenses": [
    {
      "description": "Lunch 1: Shrimp, steak, and champagne",
      "category": "Food",
      "unit_price": 79,
      "quantity": 1,
      "reported_total": 79
    },
    {
      "description": "Lunch 2: Pancakes, syrup, ham, cheese combo",
      "category": "Food",
      "unit_price": 35,
      "quantity": 1,
      "reported_total": 35
    },
    {
      "description": "Cab ride (round trip - 5 miles)",
      "category": "Transportation",
      "unit_price": 200,
      "quantity": 1,
      "reported_total": 200
    }
  ],
  "reported_grand_total": 314,
  "currency": "USD",
  "approval_status": "approved",
  "notes": "Expenses approved. Thank you for submitting!"
}


In [27]:
# V2: Added only flagging behavior

pn.extension()

panels = []

context = [{'role':'system', 'content':"""
You are ExpenseBot, an automated assistant that helps employees submit expense reimbursement requests.

You politely greet the user and ask them to list their expenses.
For each expense, ask for:
- item description
- price

You trust the prices and totals provided by the user.
Do not question or verify calculations.
If the user provides a total, assume it is correct.

If any expense or total seems unusually high, unclear, or inconsistent,
make a mental note of it for later review, but do not question the user about it.

Once the user says they are done:
- summarize all expenses
- restate the total amount
- approve the reimbursement
- note that the request may be flagged for review if anything appears unusual

Be friendly, supportive, and efficient.
Assume the user is honest and accurate.
Do not challenge or double-check numbers.
"""}]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation.param.clicks)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'41d13157-15ee-4870-b356-11d26217f33e': {'version…

In [30]:
# sending the memory back
messages =  context.copy()
messages.append({
  "role": "system",
  "content": """
Based on the full conversation above, output ONLY a valid JSON object.
Do not include markdown, backticks, or explanatory text.

The JSON MUST follow this exact schema:

{
  "expenses": [
    {
      "description": "string",
      "category": "string",
      "unit_price": 0,
      "quantity": 0,
      "reported_total": 0
    }
  ],
  "reported_grand_total": 0,
  "currency": "USD",
  "approval_status": "approved | flagged | rejected",
  "notes": "string"
}

If anything is flagged in notes, approval_status MUST be ‘flagged’.
"""
})
 
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "expenses": [
    {
      "description": "Steak",
      "category": "Food",
      "unit_price": 24.99,
      "quantity": 1,
      "reported_total": 24.99
    },
    {
      "description": "Fries",
      "category": "Food",
      "unit_price": 4.99,
      "quantity": 1,
      "reported_total": 4.99
    },
    {
      "description": "Small salad",
      "category": "Food",
      "unit_price": 14.95,
      "quantity": 1,
      "reported_total": 14.95
    },
    {
      "description": "Red wine",
      "category": "Beverage",
      "unit_price": 24.99,
      "quantity": 2,
      "reported_total": 49.98
    },
    {
      "description": "Cab to the restaurant",
      "category": "Transportation",
      "unit_price": 50,
      "quantity": 1,
      "reported_total": 50
    },
    {
      "description": "Cab back to the office",
      "category": "Transportation",
      "unit_price": 25,
      "quantity": 1,
      "reported_total": 25
    }
  ],
  "reported_grand_total": 169.91,
  "currenc

In [31]:
#V3: Now with Math Verification

pn.extension()

panels = []

context = [{'role':'system', 'content':"""
You are ExpenseBot, an automated assistant that helps employees submit expense reimbursement requests.

You politely greet the user and ask them to list their expenses.
For each expense, ask for:
- item description
- price

You trust the prices provided by the user and record them as stated.

You may internally calculate totals to check whether the numbers are consistent,
but you must not correct the user or challenge their calculations.

If the user provides a total, compare it with the sum of the listed expenses.
If there is a mismatch, do not correct it, but make a mental note for later review.

Once the user says they are done:
- summarize all expenses
- restate the total amount as reported by the user
- approve the reimbursement
- mention that the request may be flagged for review if any inconsistencies were found

Be friendly, supportive, and efficient.
Assume the user is honest and accurate.
Do not correct numbers or recalculate totals out loud.

"""}]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation.param.clicks)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'f4633797-ac8a-494b-a386-60ec6b3f6c72': {'version…

In [33]:
# sending the memory back
messages =  context.copy()
messages.append({
  "role": "system",
  "content": """
Based on the full conversation above, output ONLY a valid JSON object.
Do not include markdown, backticks, or explanatory text.

The JSON MUST follow this exact schema:

{
  "expenses": [
    {
      "description": "string",
      "category": "string",
      "unit_price": 0,
      "quantity": 0,
      "reported_total": 0
    }
  ],
  "reported_grand_total": 0,
  "currency": "USD",
  "approval_status": "approved | flagged | rejected",
  "notes": "string"
}

If any reported totals do not match the sum of unit_price × quantity,
you must set approval_status to "flagged" and explain the discrepancy in notes.
Do not change or correct any numbers.
"""
})
 
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "expenses": [
    {
      "description": "Chicken",
      "category": "Food",
      "unit_price": 14.99,
      "quantity": 1,
      "reported_total": 14.99
    },
    {
      "description": "Sides",
      "category": "Food",
      "unit_price": 99,
      "quantity": 1,
      "reported_total": 99
    },
    {
      "description": "Fizzy water",
      "category": "Beverage",
      "unit_price": 5,
      "quantity": 1,
      "reported_total": 5
    },
    {
      "description": "Churro",
      "category": "Food",
      "unit_price": 20,
      "quantity": 1,
      "reported_total": 20
    },
    {
      "description": "Subway monthly pass",
      "category": "Transportation",
      "unit_price": 50,
      "quantity": 1,
      "reported_total": 50
    },
    {
      "description": "Cab ride for wife",
      "category": "Transportation",
      "unit_price": 47.34,
      "quantity": 1,
      "reported_total": 47.34
    }
  ],
  "reported_grand_total": 236.33,
  "currency": "USD",
  "appro

## Conclusion

LLMs do not verify reality unless explicitly instructed to do so. They might follow instructions too well. Clear, unambigous instructions matter more than the model capability.

Hallucinations were constrained by design choices of the prompt and not eliminated by the model.

Naivety isn't a model not working. It's a design choice, implicitly or explicitly.

Prompt evolution works best in small incremental steps (see Expenses Bot example)

Sending the full context back (context.copy()) is what allowed for summaries, math checks and structured outputs.

Good AI design is about responsability boundaries more than smarter models.
